In [20]:
import pandas as pd
import numpy as np
import math
import scipy.stats as st
import networkx as nx
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder

ModuleNotFoundError: No module named 'sklearn'

In [12]:
# Load dataset
data = pd.read_csv('titanic_data.csv')
features = [Str for Str in data.columns]
Data = [data[LIST].tolist() for LIST in features]

In [19]:
# One-Hot Encoding
CATEGORICAL_FEATURES = ['Pclass', 'Sex', 'Siblings/Spouses Aboard', 'Parents/Children Aboard']
NUMERICAL_FEATURES = ['Age', 'Fare']


